## Goal : find french and english comments in a given set of comments

In [1]:
# packages used afterwards
import pandas as pd
import numpy as np
from Language_Detection import *

In [ ]:
# Load the dataframe containing the comments to classify

#list = [['et oui mais bon je voudrais pas dire dass ich nicht hier bin, weil du','Hello there, what are you doing?','Bonjour, eh oui c\'est pas poli','Check out TalkToMeInKorean. They at least HAD something like that, don\'t know their current product offering.','als du es dir vorgestellt hast.','Don\'t have to time to try it now, but I love your website (and app) design. Nice popping colors.'],[-1,-1,-1,-1,-1,-1]]
#initial_df = pd.DataFrame(list, index=['body', 'body_lang']).T
initial_df = pd.read_pickle('Data/processed_comments')


## Language classification using the package langdetect

In [ ]:
# Perform langdetect classification (=python package)
langdetect_classification_df = langdetect_dataframe(initial_df,seed=4)
langdetect_classification_df.head(50)

In [ ]:
# Save it to a pickle file
langdetect_classification_df.to_pickle('Data/Classified/langdetect_classification')

In [ ]:
langdetect_classification_df[['body_lang']].value_counts()

## Remove undefined featuring urls and then perform a new language detection  

In [2]:
# read file saved after having run the previous part 
df = pd.read_pickle('Data/Classified/langdetect_classification')

In [3]:
# Remove all hyperlinks 
links = (df['body'].str.contains('http')) & (df['body_lang']=='U')
df1 = df[~links]

In [ ]:
# Run the package detectlang again, only on undefined comments, with another seed
df2 = langdetect_dataframe(df1[df1['body_lang']=='U'],seed = 0)

In [4]:
# Save it to a pickle file
df1.to_pickle('Data/Classified/langdetect_classification_1')
df2.to_pickle('Data/Classified/langdetect_classification_2')

NameError: name 'df2' is not defined

In [5]:
# Read the file we saved before
df2= pd.read_pickle('Data/Classified/langdetect_classification_2')

In [6]:
# Put the newly classified comments in the previous dataframe
df3 = df2.combine_first(df1)

In [7]:
df3[['body_lang']].value_counts()

body_lang
en           633544
N             70993
fr            11253
U              8035
dtype: int64

In [21]:
df4_fr = df3[df3['body_lang']=='fr']
df4_en = df3[df3['body_lang']=='en']
df4_other = df3[df3['body_lang']=='N']
df4_undef = df3[df3['body_lang']=='U']

In [18]:
N = df1[['author']].nunique().item()
df_sum_fr = df4_fr.groupby('author').number_of_words.agg('sum')
df_sum_en = df4_en.groupby('author').number_of_words.agg('sum')
print("There were ",N," authors with >5000 words")
print("And after this classification, ")
print((df_sum_fr >= 5000).sum(), "  for french")
print((df_sum_en >= 5000).sum(), " for english")

There were  980  authors with >5000 words
And after this classification, 
14   for french
954  for english


In [22]:
# Save all frames to pickle files
df3.to_pickle('Data/Classified/langdetect_classification_3')
df4_fr.to_pickle('Data/Classified/french_comments')
df4_en.to_pickle('Data/Classified/english_comments')
df4_other.to_pickle('Data/Classified/other_language_comments')
df4_undef.to_pickle('Data/Classified/undefined_comments')


## Human classification ##

In [ ]:
# Classify by hand the undefined comments (where the package langdetect couldn't provide a satisfying answer)
human_classification_df = human_class_df(langdetect_classification_df)
human_classification_df.head(10)

In [ ]:
# save it to a csv file
human_classification_df.to_csv('Data/Classified/human_classification.csv')